In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

# Fonction pour redimensionner l'image tout en préservant le rapport d'aspect
def resize_image_to_square(image, image_size=(1024, 1024)):
    h, w, _ = image.shape
    new_h, new_w = image_size
    scale = min(new_h / h, new_w / w)
    new_dim = (int(w * scale), int(h * scale))
    
    resized_image = cv2.resize(image, new_dim, interpolation=cv2.INTER_AREA)
    
    # Créer un fond carré blanc et placer l'image redimensionnée au centre
    square_image = np.ones((new_h, new_w, 3), dtype=np.uint8) * 255  # Fond blanc
    top = (new_h - resized_image.shape[0]) // 2
    left = (new_w - resized_image.shape[1]) // 2
    square_image[top:top + resized_image.shape[0], left:left + resized_image.shape[1]] = resized_image
    
    return square_image

# Charger le module de transfert de style de TensorFlow Hub
style_url = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(style_url)

# Charger l'image de style depuis le chemin local
style_image_path = r"C:\Users\vPro\OneDrive\Bureau\van.png"  # Remplacez ce chemin avec votre image de style
style_image = cv2.imread(style_image_path)  # Charger l'image de style

if style_image is None:
    print("Error: Could not load style image. Please check the path.")
    exit()
else:
    print("Style image loaded successfully.")

# Redimensionner l'image de style
frame_size = 1024
resized_style_image = resize_image_to_square(style_image, image_size=(frame_size, frame_size))

# Initialiser la capture vidéo
cap = cv2.VideoCapture(0)  # 0 pour la caméra intégrée, essayez 1 si cela ne fonctionne pas

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

while cap.isOpened():
    # Lire une image de la caméra
    flag, frame = cap.read()
    if not flag:
        print("Error: Could not capture frame from camera.")
        break
    
    # Convertir et redimensionner l'image de contenu
    image_rgb_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_image_np = resize_image_to_square(image_rgb_np, image_size=(frame_size, frame_size))
    
    # Appliquer le transfert de style uniquement si les images sont valides
    try:
        # Convertir les images en float32 avant le transfert de style
        resized_image_np = tf.image.convert_image_dtype(resized_image_np, dtype=tf.float32)
        resized_style_image = tf.image.convert_image_dtype(resized_style_image, dtype=tf.float32)

        # Effectuer le transfert de style
        outputs = hub_module(resized_image_np[tf.newaxis, ...], resized_style_image[tf.newaxis, ...])
        stylized_image = outputs[0]

        # Convertir le résultat en image affichable
        image_pil = tf.keras.preprocessing.image.array_to_img(stylized_image[0])
        image_bgr_np = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        
        # Afficher l'image stylisée
        cv2.imshow("Style Transfer", image_bgr_np)
    except Exception as e:
        print(f"Error during style transfer: {e}")
    
    # Quitter la boucle si la touche 'q' est pressée
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Libérer les ressources
cap.release()
cv2.destroyAllWindows()
